In [6]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import sys

# 1. define nn
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        """
        Initializes the neural network layers.
        input_dim: The number of input features.
        """
        super(SimpleNN, self).__init__()
        # Input layer to 1st hidden layer
        self.layer_1 = nn.Linear(input_dim, 64)
        self.relu_1 = nn.ReLU()
        
        # 1st hidden layer to 2nd hidden layer
        self.layer_2 = nn.Linear(64, 32)
        self.relu_2 = nn.ReLU()
        
        # 2nd hidden layer to Output layer
        self.output_layer = nn.Linear(32, 1)

    def forward(self, x):
        """
        Defines the forward pass of the network.
        """
        x = self.layer_1(x)
        x = self.relu_1(x)
        x = self.layer_2(x)
        x = self.relu_2(x)
        x = self.output_layer(x) # gives raw logits
        return x

# 2. preprocessing
print("Loading data...")
try:
    data = pd.read_csv('data.csv')
    print("Data loaded. Displaying first 5 rows:")
    print(data.head())

except FileNotFoundError:
    print("Error: 'data.csv' not found.")
    print("Please make sure 'data.csv' is in the same directory as your .ipynb file.")
    # stop execution in case of file not found error
    sys.exit()

# usless columns hatao
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# One-Hot Encoding for category featues
data = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)

# Target 'y' is what we want to predict ('Exited')
y = data['Exited']
# Features 'X' are all columns *except* 'Exited'
X = data.drop('Exited', axis=1)

#no. of input featues
input_dim = X.shape[1]

# 4. 80-20 split
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.2, random_state=42)

# Calculate weights based *only* on the training data
class_counts = y_train_df.value_counts()
weight_for_0 = class_counts[1] / class_counts[0]
weight_for_1 = class_counts[0] / class_counts[1]

print(f"\nTraining data shows {class_counts[0]} 'Not Churned' and {class_counts[1]} 'Churned'.")
print(f"Calculated positive class weight (for 'Churned'): {weight_for_1:.2f}")

# Create a tensor for the positive class weight
pos_weight = torch.tensor([weight_for_1], dtype=torch.float32)

#scaling
scaler = StandardScaler()
numerical_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
X_train_df[numerical_cols] = scaler.fit_transform(X_train_df[numerical_cols])
X_test_df[numerical_cols] = scaler.transform(X_test_df[numerical_cols])

#making tensors from data
X_train = torch.tensor(X_train_df.values.astype(float), dtype=torch.float32)
y_train = torch.tensor(y_train_df.values.astype(float), dtype=torch.float32).view(-1, 1)

X_test = torch.tensor(X_test_df.values.astype(float), dtype=torch.float32)
y_test = torch.tensor(y_test_df.values.astype(float), dtype=torch.float32).view(-1, 1)

model = SimpleNN(input_dim)

# This is more numerically stable and lets us pass the class weight
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

#adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

#training
print("\nTraining the model (with class weights)...")
num_epochs = 100
batch_size = 32

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs) # outputs are raw logits
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training finished.")

print("\nEvaluating model on test data...")
model.eval()

with torch.no_grad():
    # Get raw logits from the model
    y_pred_logits = model(X_test)
    
    # Apply sigmoid to convert logits to probabilities
    y_pred_probs = torch.sigmoid(y_pred_logits)
    
    # Convert probabilities (0 to 1) to binary class (0 or 1)
    y_pred_binary = (y_pred_probs > 0.5).float()
    
    y_test_np = y_test.numpy()
    y_pred_binary_np = y_pred_binary.numpy()

    accuracy = accuracy_score(y_test_np, y_pred_binary_np)
    print(f'Accuracy: {accuracy * 100:.2f}%')
    
    print("\nClassification Report:")
    try:
        print(classification_report(y_test_np, y_pred_binary_np, target_names=['Not Churned (0)', 'Churned (1)'], zero_division=0))
    except ValueError as e:
        print(f"Could not generate full report (this is common with tiny sample data): {e}")

Loading data...
Data loaded. Displaying first 5 rows:
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.5